<a href="https://colab.research.google.com/github/anhelina-bond/Image-Filter-Generator/blob/main/ImageFilterGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 15.4 MB/s eta 

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-p5rl511i
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-p5rl511i
  Resolved https://github.com/tensorflow/examples.git to commit b5a8265e0b655001eaa859e7bd8ac9b4e03f3ce8
  Preparing metadata (setup.py) ... done


In [ ]:
!nvidia-smi

Tue Jan  6 18:02:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# ==============================================================================
# SECTION 1: SETUP AND CONFIGURATION
# ==============================================================================
import tensorflow as tf
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import gc
import keras

print("TensorFlow Version:", tf.__version__)

AUTOTUNE = tf.data.AUTOTUNE

# --- Mount Drive and define all paths ---
from google.colab import drive
drive.mount('/content/drive')
DRIVE_ROOT = '/content/drive/MyDrive/ImageFilterGenerator_Regression' # New Project Folder
CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, 'checkpoints')
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# --- Model and Data Parameters ---
BATCH_SIZE = 32 # Can use a larger batch size for regression
IMG_WIDTH = 256
IMG_HEIGHT = 256
NUM_PARAMETERS =14 # This MUST match the output of your data function


TensorFlow Version: 2.19.0
Mounted at /content/drive


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

print("\n--- Loading COCO-2017 as the content dataset... ---")
content_dataset = foz.load_zoo_dataset("coco-2017", split="train", max_samples=1000, shuffle=True, drop_existing_dataset=True)
content_filepaths = content_dataset.values("filepath")
print(f"Loaded {len(content_filepaths)} content images.")
content_dataset.delete() # Free up memory


--- Loading COCO-2017 as the content dataset... ---


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


340 images found; downloading the remaining 660


...

 100% |███████████████| 1000/1000 [7.1s elapsed, 0s remaining, 148.8 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [7.1s elapsed, 0s remaining, 148.8 samples/s]      


Dataset 'coco-2017-train-1000' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-train-1000' created


Loaded 1000 content images.


In [ ]:
def load_image(image_file):
    image_bytes = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image_bytes, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(image, tf.float32) / 255.0

def apply_and_return_params_v2(image):
    params = {}
    delta = tf.random.uniform([], -0.15, 0.15); params['brightness'] = (delta + 0.15) / 0.3
    image = tf.image.adjust_brightness(image, delta)
    sat_factor = tf.random.uniform([], 0.7, 1.3); params['saturation'] = (sat_factor - 0.7) / 0.6
    image = tf.image.adjust_saturation(image, sat_factor)
    gains = tf.random.uniform([3], 0.9, 1.1); params['r_gain'] = (gains[0] - 0.9) / 0.2
    params['g_gain'] = (gains[1] - 0.9) / 0.2; params['b_gain'] = (gains[2] - 0.9) / 0.2
    image = image * gains
    if tf.random.uniform([]) > 0.5:
        factor = tf.random.uniform([], 2.0, 8.0); params['s_curve_contrast'] = (factor - 2.0) / 6.0
        image = tf.sigmoid(factor * (image - 0.5)); params['linear_contrast'] = 0.5
    else:
        factor = tf.random.uniform([], 0.8, 1.2); params['linear_contrast'] = (factor - 0.8) / 0.4
        image = tf.image.adjust_contrast(image, factor); params['s_curve_contrast'] = 0.0
    if tf.random.uniform([]) > 0.5:
        shadow_hue = tf.random.uniform([], 0.4, 0.7); shadow_saturation = tf.random.uniform([], 0.0, 0.25)
        highlight_hue = tf.random.uniform([], 0.05, 0.15); highlight_saturation = tf.random.uniform([], 0.0, 0.25)
        params['shadow_hue'] = (shadow_hue - 0.4) / 0.3; params['shadow_sat'] = shadow_saturation / 0.25
        params['highlight_hue'] = (highlight_hue - 0.05) / 0.1; params['highlight_sat'] = highlight_saturation / 0.25
    else:
        params['shadow_hue'] = 0.5; params['shadow_sat'] = 0.0; params['highlight_hue'] = 0.5; params['highlight_sat'] = 0.0
    if tf.random.uniform([]) > 0.5:
        strength = tf.random.uniform([], 0.0, 0.05); params['grain_strength'] = strength / 0.05
        image = image + tf.random.normal(shape=tf.shape(image), stddev=strength)
    else: params['grain_strength'] = 0.0
    if tf.random.uniform([]) > 0.5:
        shape = tf.shape(image); y, x = tf.meshgrid(tf.linspace(-1.0, 1.0, shape[0]), tf.linspace(-1.0, 1.0, shape[1]))
        radius = tf.sqrt(x**2 + y**2)
        strength = tf.random.uniform([], 0.5, 1.5); params['vignette_strength'] = (strength - 0.5) / 1.0
        falloff = tf.random.uniform([], 0.5, 1.5); params['vignette_falloff'] = (falloff - 0.5) / 1.0
        image = image * tf.expand_dims(tf.clip_by_value(1.0 - strength * (radius ** falloff), 0.0, 1.0), axis=-1)
    else: params['vignette_strength'] = 0.0; params['vignette_falloff'] = 0.0
    image = tf.clip_by_value(image, 0.0, 1.0)
    param_vector = tf.stack([
        params['brightness'], params['saturation'], params['r_gain'], params['g_gain'], params['b_gain'],
        params['s_curve_contrast'], params['linear_contrast'], params['shadow_hue'], params['shadow_sat'],
        params['highlight_hue'], params['highlight_sat'], params['grain_strength'],
        params['vignette_strength'], params['vignette_falloff']
    ])
    return image, param_vector

def create_regression_training_pair(image_file):
    neutral_image = load_image(image_file)
    filtered_image, param_vector = apply_and_return_params_v2(neutral_image)
    return filtered_image, param_vector # Input: Filtered Image, Label: Parameter Vector

# --- Create the final training dataset ---
print("\n--- Building Data Pipeline... ---")
train_ds = tf.data.Dataset.from_tensor_slices(content_filepaths)
train_ds = train_ds.map(create_regression_training_pair, num_parallel_calls=AUTOTUNE)
# No caching needed for this approach as data is generated on the fly
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
del content_filepaths
gc.collect()
print("✅ Data pipeline created.")


--- Building Data Pipeline... ---
✅ Data pipeline created.


In [ ]:
# ==============================================================================
# SECTION 3: PARAMETER REGRESSION MODEL ARCHITECTURE (Corrected & Simplified)
# ==============================================================================
import keras

@keras.saving.register_keras_serializable()
class ParameterRegressor(tf.keras.Model):
    def __init__(self, num_parameters, is_fine_tuning=False, **kwargs):
        super().__init__(**kwargs)

        self.num_parameters = num_parameters
        self.is_fine_tuning = is_fine_tuning

        self.base_model = tf.keras.applications.EfficientNetB0(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
            include_top=False,
            weights='imagenet'
        )
        self.base_model.trainable = is_fine_tuning
        if is_fine_tuning:
            for layer in self.base_model.layers[:-20]:
                layer.trainable = False

        # Define all layers
        self.rescale = tf.keras.layers.Rescaling(255.0)
        self.preprocess = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input)
        self.pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dense1 = tf.keras.layers.Dense(512, activation='swish')
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense2 = tf.keras.layers.Dense(256, activation='swish')
        self.output_layer = tf.keras.layers.Dense(num_parameters, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.rescale(inputs)
        x = self.preprocess(x)
        x = self.base_model(x, training=training)
        x = self.pooling(x)
        x = self.batch_norm(x, training=training)
        x = self.dense1(x)
        x = self.dropout(x, training=training)
        x = self.dense2(x)
        return self.output_layer(x)

    def get_config(self):
        base_config = super().get_config()
        base_config.update(
            {
                "num_parameters": self.num_parameters,
                "is_fine_tuning": self.is_fine_tuning,
            }
        )
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [ ]:
# ==============================================================================
# --- NEW: SECTION FOR CALLBACKS ---
# ==============================================================================
import keras

PARAM_ORDER = [
    'brt', 'sat', 'r', 'g', 'b', 's_crv', 'l_crv', 'sh_h', 'sh_s',
    'hi_h', 'hi_s', 'grain', 'vig_s', 'vig_f'
]

# Get a single batch from dataset to use for predictions
sample_batch_for_callback = next(iter(train_ds))
filtered_sample, true_params_sample = sample_batch_for_callback

# Custom Callback Class
class PredictionCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        predicted_params = self.model.predict(filtered_sample, verbose=0)

        clear_output(wait=True)

        print(f"\n--- Prediction Sample at Epoch {epoch + 1} ---")
        print(" " * 20 + " ".join([f"{p:<5}" for p in PARAM_ORDER]))

        # Display the first sample in the batch
        print("True Parameters:      " + " ".join([f"{p:.2f}" for p in true_params_sample[0].numpy()]))
        print("Predicted Parameters: " + " ".join([f"{p:.2f}" for p in predicted_params[0]]))
        print("-" * 80)


In [ ]:
import os
import glob
def find_latest_h5_checkpoint(directory):
    h5_files = glob.glob(os.path.join(directory, '*.weights.h5'))
    if not h5_files:
        return None
    latest_h5_file = sorted(h5_files)[-1]
    return latest_h5_file

In [ ]:
# ==============================================================================
# PHASE 1: TRAIN THE HEAD
# ==============================================================================
print("\n--- STARTING PHASE 1: Training the Head ---")
PHASE_1_EPOCHS = 50
CHECKPOINT_DIR_PHASE1 = os.path.join(DRIVE_ROOT, 'checkpoints_phase1')

import glob

# Build the model for Phase 1
phase1_model = ParameterRegressor(num_parameters=14, is_fine_tuning=False)
phase1_model.base_model.trainable = False

# Compile the model
phase1_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError()
)

phase1_model.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, 3))

#SETUP CHECKPOINTING AND RESUMPTION FOR PHASE 1
checkpoint_path_p1 = os.path.join(CHECKPOINT_DIR_PHASE1, "ckpt-epoch{epoch:04d}.weights.h5")

# Create the ModelCheckpoint callback
checkpoint_callback_p1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_p1,
    save_weights_only=True,
    save_freq='epoch',
    verbose=1 # Prints a message when it saves
)

# Find the latest checkpoint if one exists
latest_checkpoint_p1 = find_latest_h5_checkpoint(CHECKPOINT_DIR_PHASE1)
initial_epoch_p1 = 0
if latest_checkpoint_p1:
    print(f"Phase 1 H5 checkpoint found: {latest_checkpoint_p1}. Resuming training.")
    phase1_model.load_weights(latest_checkpoint_p1)
    initial_epoch_p1 = int(latest_checkpoint_p1.split('epoch')[1].split('.')[0])
else:
    print("No Phase 1 checkpoint found. Starting from scratch.")

# Train using
history_phase1 = phase1_model.fit(
    train_ds,
    epochs=PHASE_1_EPOCHS,
    callbacks=[PredictionCallback(), checkpoint_callback_p1],
    initial_epoch=initial_epoch_p1
)

print(f"✅ Phase 1 training complete.")


# ==============================================================================
# PHASE 2: FINE-TUNING
# ==============================================================================
print("\n--- STARTING PHASE 2: Fine-Tuning ---")
PHASE_2_EPOCHS = 200
CHECKPOINT_DIR_PHASE2 = os.path.join(DRIVE_ROOT, 'checkpoints_phase2')

# Build the model for Phase 2
phase2_model = ParameterRegressor(num_parameters=14, is_fine_tuning=True)
phase2_model.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, 3))

# Load the weights from the end of Phase 1 as a starting point
phase1_final_weights = find_latest_h5_checkpoint(CHECKPOINT_DIR_PHASE1)
if phase1_final_weights:
    print(f"Loading weights from Phase 1 H5 checkpoint: {phase1_final_weights}")
    phase2_model.load_weights(phase1_final_weights)
    print("✅ Weights from Phase 1 loaded successfully.")
else:
    print("WARNING: Could not find Phase 1 weights.")

# Uunfreeze the top layers of the base model.
phase2_model.base_model.trainable = True
for layer in phase2_model.base_model.layers[:-20]:
    layer.trainable = False

# Re-compile the model with a low learning rate.
phase2_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.MeanSquaredError()
)

# SETUP CHECKPOINTING AND RESUMPTION FOR PHASE 2
checkpoint_path_p2 = os.path.join(CHECKPOINT_DIR_PHASE2, "ckpt-epoch{epoch:04d}.weights.h5")
checkpoint_callback_p2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_p2,
    save_weights_only=True,
    save_freq='epoch',
    verbose=1
)

latest_checkpoint_p2 = find_latest_h5_checkpoint(CHECKPOINT_DIR_PHASE2)
initial_epoch_p2 = 0
if latest_checkpoint_p2:
    print(f"Phase 2 H5 checkpoint found: {latest_checkpoint_p2}. Resuming fine-tuning.")
    phase2_model.load_weights(latest_checkpoint_p2)
    initial_epoch_p2 = int(latest_checkpoint_p2.split('epoch')[1].split('.')[0])
else:
    print("No Phase 2 checkpoint found. Starting fine-tuning from the beginning.")

# Continue training
history_phase2 = phase2_model.fit(
    train_ds,
    epochs=PHASE_2_EPOCHS,
    callbacks=[PredictionCallback(), checkpoint_callback_p2],
    initial_epoch=initial_epoch_p2
)

# Save final model
print("\n--- Preparing to save the final model ---")
FINAL_MODEL_PATH = os.path.join(DRIVE_ROOT, 'final_regression_model.keras')

print("Step 1: Creating a fresh model instance for saving...")
final_model_to_save = ParameterRegressor(num_parameters=14, is_fine_tuning=True)

print("Step 2: Building the fresh model instance...")
dummy_input = tf.zeros([1, AI_INPUT_HEIGHT, AI_INPUT_WIDTH, 3])
_ = final_model_to_save(dummy_input)

print("Step 3: Finding the latest checkpoint from Phase 2...")
latest_finetune_checkpoint = find_latest_h5_checkpoint(CHECKPOINT_DIR_PHASE2)
if not latest_finetune_checkpoint:
    raise FileNotFoundError("Could not find any Phase 2 checkpoints to save.")

print(f"Step 4: Loading weights from {latest_finetune_checkpoint}...")
final_model_to_save.load_weights(latest_finetune_checkpoint)

print(f"Step 5: Saving the final, clean model to {FINAL_MODEL_PATH}...")
final_model_to_save.save(FINAL_MODEL_PATH)

print(f"✅ Final model saved successfully.")



--- STARTING PHASE 1: Training the Head ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'parameter_regressor_6', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Phase 1 H5 checkpoint found: /content/drive/MyDrive/ImageFilterGenerator_Regression/checkpoints_phase1/ckpt-epoch0050.weights.h5. Resuming training.
✅ Phase 1 training complete.

--- STARTING PHASE 2: Fine-Tuning ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'parameter_regressor_7', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Loading weights from Phase 1 H5 checkpoint: /content/drive/MyDrive/ImageFilterGenerator_Regression/checkpoints_phase1/ckpt-epoch0050.weights.h5
✅ Weights from Phase 1 loaded successfully.
Phase 2 H5 checkpoint found: /content/drive/MyDrive/ImageFilterGenerator_Regression/checkpoints_phase2/ckpt-epoch0200.weights.h5. Resuming fine-tuning.

--- Preparing to save the final model ---
Step 1: Creating a fresh model instance for saving...
Step 2: Building the fresh model instance...
Step 3: Finding the latest checkpoint from Phase 2...
Step 4: Loading weights from /content/drive/MyDrive/ImageFilterGenerator_Regression/checkpoints_phase2/ckpt-epoch0200.weights.h5...
Step 5: Saving the final, clean model to /content/drive/MyDrive/ImageFilterGenerator_Regression/final_regression_model.keras...
✅ Final model saved successfully.


In [ ]:
# ==============================================================================
# SECTION 7: POST-TRAINING INFERENCE & APPLICATION (Final Corrected Version)
# ==============================================================================
print("\n--- Preparing Final Inference Application ---")

import ipywidgets as widgets
from google.colab import files
import gradio as gr
import nest_asyncio
import cv2
import numpy as np
import tensorflow as tf
import keras

nest_asyncio.apply()

# Define Global Paths and Constants
DRIVE_ROOT = '/content/drive/MyDrive/ImageFilterGenerator_Regression'
FINAL_MODEL_PATH = os.path.join(DRIVE_ROOT, 'final_regression_model.keras')
PARAM_ORDER = [
    'brightness', 'saturation', 'r_gain', 'g_gain', 'b_gain', 's_curve_contrast',
    'linear_contrast', 'shadow_hue', 'shadow_sat', 'highlight_hue', 'highlight_sat',
    'grain_strength', 'vignette_strength', 'vignette_falloff'
]
NUM_PARAMETERS = 14
AI_INPUT_HEIGHT = 256
AI_INPUT_WIDTH = 256


@keras.saving.register_keras_serializable()
class ParameterRegressor(tf.keras.Model):
    def __init__(self, num_parameters, is_fine_tuning=False, **kwargs):
        super().__init__(**kwargs)
        self.num_parameters = num_parameters
        self.is_fine_tuning = is_fine_tuning

        self.base_model = tf.keras.applications.EfficientNetB0(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
            include_top=False,
            weights='imagenet'
        )

        self.rescale = tf.keras.layers.Rescaling(255.0)
        self.preprocess = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input)
        self.pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dense1 = tf.keras.layers.Dense(512, activation='swish')
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense2 = tf.keras.layers.Dense(256, activation='swish')
        self.output_layer = tf.keras.layers.Dense(num_parameters, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.rescale(inputs)
        x = self.preprocess(x)

        x = self.base_model(x, training=training)
        x = self.pooling(x)
        x = self.batch_norm(x, training=training)
        x = self.dense1(x)
        x = self.dropout(x, training=training)
        x = self.dense2(x)
        return self.output_layer(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_parameters": self.num_parameters,
            "is_fine_tuning": self.is_fine_tuning,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

def apply_params_to_image(image, param_vector):
    """
    Takes a [0,1] image tensor of ANY size and applies the predicted filter.
    This is the "parameter decoder".
    """
    params = dict(zip(PARAM_ORDER, param_vector))

    # De-normalize and apply each effect mathematically
    image = tf.image.adjust_brightness(image, params['brightness'] * 0.3 - 0.15)
    image = tf.image.adjust_saturation(image, params['saturation'] * 0.6 + 0.7)
    gains = tf.constant([params['r_gain'] * 0.2 + 0.9, params['g_gain'] * 0.2 + 0.9, params['b_gain'] * 0.2 + 0.9], dtype=tf.float32)
    image *= gains

    if params['s_curve_contrast'] > 0.01:
        factor = params['s_curve_contrast'] * 6.0 + 2.0
        image = tf.sigmoid(factor * (image - 0.5))
    elif params['linear_contrast'] > 0.51 or params['linear_contrast'] < 0.49:
        factor = params['linear_contrast'] * 0.4 + 0.8
        image = tf.image.adjust_contrast(image, factor)

    if params['grain_strength'] > 0.01:
        strength = params['grain_strength'] * 0.05
        image += tf.random.normal(shape=tf.shape(image), stddev=strength)

    if params['vignette_strength'] > 0.01:

        def apply_vignette_numpy(image_np, strength_param, falloff_param):
            height, width, _ = image_np.shape

            x_coords = np.linspace(-1.0, 1.0, width)
            y_coords = np.linspace(-1.0, 1.0, height)
            x, y = np.meshgrid(x_coords, y_coords)

            radius = np.sqrt(x**2 + y**2)

            # De-normalize params inside the function
            strength = strength_param * 1.0 + 0.5
            falloff = falloff_param * 1.0 + 0.5

            vignette_mask = np.clip(1.0 - strength * (radius ** falloff), 0.0, 1.0)

            vignette_3_channel = np.stack([vignette_mask] * 3, axis=-1)

            return image_np * vignette_3_channel
        image = tf.py_function(
            func=apply_vignette_numpy,
            inp=[image, params['vignette_strength'], params['vignette_falloff']],
            Tout=tf.float32
        )

    return tf.clip_by_value(image, 0.0, 1.0)

def create_xmp_preset(param_vector, preset_name="GeneratedPreset"):
    """Generates an XMP file content string with all 14 parameters mapped."""
    params = dict(zip(PARAM_ORDER, param_vector))

    exposure = (params['brightness'] * 0.6 - 0.3) * 5 # Range approx -1.5 to +1.5
    contrast = (params['linear_contrast'] * 80) - 40   # Range approx -40 to +40
    saturation = (params['saturation'] * 120) - 60     # Range approx -60 to +60

    # Tone Curve (approximating S-curve)
    s_curve_intensity = params['s_curve_contrast'] * 50 # 0 to 50
    highlights2012 = s_curve_intensity
    lights2012 = s_curve_intensity / 2
    darks2012 = -s_curve_intensity / 2
    shadows2012 = -s_curve_intensity

    # Color Mixer (approximating RGB gains)
    red_hue = (params['r_gain'] - 0.5) * 20
    green_hue = (params['g_gain'] - 0.5) * 20
    blue_hue = (params['b_gain'] - 0.5) * 20

    # Color Grading (Split Toning)
    shadow_hue_val = params['shadow_hue'] * 360
    shadow_sat_val = params['shadow_sat'] * 100
    highlight_hue_val = params['highlight_hue'] * 360
    highlight_sat_val = params['highlight_sat'] * 100

    # Effects Tab
    grain_amount = params['grain_strength'] * 100
    vignette_amount = (params['vignette_strength'] * -100)
    vignette_midpoint = (1.0 - params['vignette_falloff']) * 100

    xmp_template = f"""
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 5.6-c140 79.160451, 2017/05/06-01:08:21        ">
 <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <rdf:Description rdf:about="" xmlns:crs="http://ns.adobe.com/camera-raw-settings/1.0/">
   <crs:PresetName>{preset_name}</crs:PresetName>
   <crs:HasSettings>True</crs:HasSettings>

   <!-- Basic Panel -->
   <crs:Exposure2012>{exposure:.4f}</crs:Exposure2012>
   <crs:Contrast2012>{contrast:.4f}</crs:Contrast2012>
   <crs:Saturation>{saturation:.4f}</crs:Saturation>

   <!-- Tone Curve Panel -->
   <crs:ToneCurvePV2012>
    <rdf:Seq>
     <rdf:li>0, 0</rdf:li>
     <rdf:li>255, 255</rdf:li>
    </rdf:Seq>
   </crs:ToneCurvePV2012>
   <crs:Highlights2012>{highlights2012:.0f}</crs:Highlights2012>
   <crs:Lights2012>{lights2012:.0f}</crs:Lights2012>
   <crs:Darks2012>{darks2012:.0f}</crs:Darks2012>
   <crs:Shadows2012>{shadows2012:.0f}</crs:Shadows2012>

   <!-- Color Mixer Panel (Hue) -->
   <crs:HueAdjustmentRed>{red_hue:.0f}</crs:HueAdjustmentRed>
   <crs:HueAdjustmentGreen>{green_hue:.0f}</crs:HueAdjustmentGreen>
   <crs:HueAdjustmentBlue>{blue_hue:.0f}</crs:HueAdjustmentBlue>

   <!-- Color Grading Panel (Split Toning) -->
   <crs:SplitToningShadowHue>{shadow_hue_val:.0f}</crs:SplitToningShadowHue>
   <crs:SplitToningShadowSaturation>{shadow_sat_val:.0f}</crs:SplitToningShadowSaturation>
   <crs:SplitToningHighlightHue>{highlight_hue_val:.0f}</crs:SplitToningHighlightHue>
   <crs:SplitToningHighlightSaturation>{highlight_sat_val:.0f}</crs:SplitToningHighlightSaturation>

   <!-- Effects Panel -->
   <crs:GrainAmount>{grain_amount:.0f}</crs:GrainAmount>
   <crs:PostCropVignetteAmount>{vignette_amount:.0f}</crs:PostCropVignetteAmount>
   <crs:PostCropVignetteMidpoint>{vignette_midpoint:.0f}</crs:PostCropVignetteMidpoint>

  </rdf:Description>
 </rdf:RDF>
</x:xmpmeta>
"""
    return xmp_template.strip()

# Build the model architecture
print("\n--- Initializing and Loading the Final AI Model (ONCE) ---")

if not os.path.exists(FINAL_MODEL_PATH):
    raise FileNotFoundError(f"FATAL: Final model not found at {FINAL_MODEL_PATH}. Please train and save the model first.")

GLOBAL_INFERENCE_MODEL = tf.keras.models.load_model(FINAL_MODEL_PATH)

print("✅ Final model loaded successfully.")
GLOBAL_INFERENCE_MODEL.summary()

# MAIN GRADIO APPLICATION FUNCTION
def generate_filter_and_apply(filtered_image_input, neutral_image_input, preset_name, file_type):
    if filtered_image_input is None: raise gr.Error("Please upload a Filtered Image (Style).")
    if neutral_image_input is None: raise gr.Error("Please upload a Neutral Image (Content).")
    if not preset_name: preset_name = "AI_Generated_Preset"

    print("\n--- New Request Received ---")

    # Predict parameters using the global model
    print("Step 1: Predicting filter parameters...")
    img_tensor_for_ai = tf.cast(filtered_image_input, tf.float32) / 255.0
    input_for_model = tf.image.resize(img_tensor_for_ai, [AI_INPUT_HEIGHT, AI_INPUT_WIDTH])
    input_for_model = tf.expand_dims(input_for_model, 0)
    predicted_params_vector = GLOBAL_INFERENCE_MODEL(input_for_model, training=False)[0].numpy()

    # Apply filter to full-resolution neutral image
    print("Step 2: Applying filter...")
    neutral_tensor_full_res = tf.cast(neutral_image_input, tf.float32) / 255.0
    result_image_tensor = apply_params_to_image(neutral_tensor_full_res, predicted_params_vector)
    result_image_np = (result_image_tensor.numpy() * 255).astype(np.uint8)

    # Generate the requested preset file
    print(f"Step 3: Generating {file_type} preset file...")
    if file_type == ".xmp (Lightroom/Photoshop)":
        xmp_content = create_xmp_preset(predicted_params_vector, preset_name)
        preset_filepath = f"{preset_name}.xmp"
        with open(preset_filepath, "w") as f: f.write(xmp_content)
    else:
        xmp_content = create_xmp_preset(predicted_params_vector, preset_name)
        preset_filepath = f"{preset_name}.xmp"
        with open(preset_filepath, "w") as f: f.write(xmp_content)

    param_str = "\n".join([f"{name}: {val:.3f}" for name, val in zip(PARAM_ORDER, predicted_params_vector)])

    print("--- Request Complete ---")
    return result_image_np, param_str, gr.File(value=preset_filepath, label=f"Download {preset_name}{os.path.splitext(preset_filepath)[1]}")

# LAUNCH THE GRADIO WEB UI
with gr.Blocks(title="Image Filter Generator") as demo:
    gr.Markdown("# 📸 Image Filter Generator (Full Resolution)")
    gr.Markdown("1. Upload a style image. 2. Upload a content image. 3. Name your preset. 4. Generate!")

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="numpy", label="1. Upload Your Filtered Image (Style)")
            neutral_input = gr.Image(type="numpy", label="2. Upload Your Neutral Image (Content)")

            with gr.Row():
                preset_name_input = gr.Textbox(label="3. Name Your Preset", placeholder="e.g., Moody_Film_Look")
                file_type_input = gr.Dropdown(
                    [".xmp (Lightroom/Photoshop)"],
                    value=".xmp (Lightroom/Photoshop)",
                    label="Select Preset Format"
                )

            generate_button = gr.Button("4. Generate and Apply Filter", variant="primary")

        with gr.Column(scale=1):
            image_output = gr.Image(type="numpy", label="Final Full-Resolution Result")

    with gr.Row():
        parameter_output = gr.Textbox(label="Predicted Normalized Parameters", lines=14, scale=1)
        file_output = gr.File(label="Download Preset", scale=1)

    generate_button.click(
        fn=generate_filter_and_apply,
        inputs=[image_input, neutral_input, preset_name_input, file_type_input],
        outputs=[image_output, parameter_output, file_output]
    )

print("\n--- Launching Gradio Application ---")
demo.launch(share=True, debug=True)


--- Preparing Final Inference Application ---

--- Initializing and Loading the Final AI Model (ONCE) ---
✅ Final model loaded successfully.


Model: "parameter_regressor_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 8, 8, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_23 (Rescaling)        │ (1, 256, 256, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_7 (Lambda)               │ (1, 256, 256, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (1, 1280)              │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (1, 512)               │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (1, 256)               │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (1, 14)                │         3,598 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,845,489 (18.48 MB)

 Trainable params: 4,800,906 (18.31 MB)

...


--- New Request Received ---
Step 1: Predicting filter parameters...
Step 2: Applying filter...
Step 3: Generating .xmp (Lightroom/Photoshop) preset file...
--- Request Complete ---


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


--- New Request Received ---
Step 1: Predicting filter parameters...
Step 2: Applying filter...
Step 3: Generating .xmp (Lightroom/Photoshop) preset file...
--- Request Complete ---


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error


--- New Request Received ---
Step 1: Predicting filter parameters...
Step 2: Applying filter...
Step 3: Generating .xmp (Lightroom/Photoshop) preset file...
--- Request Complete ---


In [ ]:
# ==============================================================================
# RESULTS FOR THE PROJECT REPORT
# ==============================================================================
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import fiftyone as fo
import fiftyone.zoo as foz
import pandas as pd

print("--- Starting Report Generation Script ---")

# Define Paths and Constants
DRIVE_ROOT = '/content/drive/MyDrive/ImageFilterGenerator_Regression'
FINAL_MODEL_PATH = os.path.join(DRIVE_ROOT, 'final_regression_model.keras')
RESULTS_DIR = os.path.join(DRIVE_ROOT, 'report_results')
os.makedirs(RESULTS_DIR, exist_ok=True)

PARAM_ORDER = [
    'brightness', 'saturation', 'r_gain', 'g_gain', 'b_gain', 's_curve_contrast',
    'linear_contrast', 'shadow_hue', 'shadow_sat', 'highlight_hue', 'highlight_sat',
    'grain_strength', 'vignette_strength', 'vignette_falloff'
]
NUM_PARAMETERS = 14
AI_INPUT_HEIGHT = 256
AI_INPUT_WIDTH = 256

# LOAD THE FINAL TRAINED MODEL
@keras.saving.register_keras_serializable()
class ParameterRegressor(tf.keras.Model):
    def __init__(self, num_parameters, is_fine_tuning=False, **kwargs):
        super().__init__(**kwargs)

        self.num_parameters = num_parameters
        self.is_fine_tuning = is_fine_tuning

        self.base_model = tf.keras.applications.EfficientNetB0(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
            include_top=False,
            weights='imagenet'
        )
        self.base_model.trainable = is_fine_tuning
        if is_fine_tuning:
            for layer in self.base_model.layers[:-20]:
                layer.trainable = False

        # Define all layers
        self.rescale = tf.keras.layers.Rescaling(255.0)
        self.preprocess = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input)
        self.pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dense1 = tf.keras.layers.Dense(512, activation='swish')
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense2 = tf.keras.layers.Dense(256, activation='swish')
        self.output_layer = tf.keras.layers.Dense(num_parameters, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.rescale(inputs)
        x = self.preprocess(x)
        x = self.base_model(x, training=training)
        x = self.pooling(x)
        x = self.batch_norm(x, training=training)
        x = self.dense1(x)
        x = self.dropout(x, training=training)
        x = self.dense2(x)
        return self.output_layer(x)

    def get_config(self):
        base_config = super().get_config()
        base_config.update(
            {
                "num_parameters": self.num_parameters,
                "is_fine_tuning": self.is_fine_tuning,
            }
        )
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

print(f"\nLoading final trained model from: {FINAL_MODEL_PATH}")
if not os.path.exists(FINAL_MODEL_PATH):
    raise FileNotFoundError("FATAL: Final model not found. Please train and save the model first.")
model = tf.keras.models.load_model(FINAL_MODEL_PATH)
print("✅ Model loaded successfully.")

model.compile(loss=tf.keras.losses.MeanSquaredError())
print("✅ Model recompiled for evaluation.")

# CREATE A HOLD-OUT TEST SET
print("\nCreating a hold-out test set from COCO validation split...")
test_content_dataset = foz.load_zoo_dataset("coco-2017", split="validation", max_samples=1000, shuffle=True)
test_filepaths = test_content_dataset.values("filepath")

def load_image(image_file):
    image_bytes = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image_bytes, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(image, tf.float32) / 255.0

def apply_and_return_params_v2(image):
    params = {}
    delta = tf.random.uniform([], -0.15, 0.15); params['brightness'] = (delta + 0.15) / 0.3
    image = tf.image.adjust_brightness(image, delta)
    sat_factor = tf.random.uniform([], 0.7, 1.3); params['saturation'] = (sat_factor - 0.7) / 0.6
    image = tf.image.adjust_saturation(image, sat_factor)
    gains = tf.random.uniform([3], 0.9, 1.1); params['r_gain'] = (gains[0] - 0.9) / 0.2
    params['g_gain'] = (gains[1] - 0.9) / 0.2; params['b_gain'] = (gains[2] - 0.9) / 0.2
    image = image * gains
    if tf.random.uniform([]) > 0.5:
        factor = tf.random.uniform([], 2.0, 8.0); params['s_curve_contrast'] = (factor - 2.0) / 6.0
        image = tf.sigmoid(factor * (image - 0.5)); params['linear_contrast'] = 0.5
    else:
        factor = tf.random.uniform([], 0.8, 1.2); params['linear_contrast'] = (factor - 0.8) / 0.4
        image = tf.image.adjust_contrast(image, factor); params['s_curve_contrast'] = 0.0
    if tf.random.uniform([]) > 0.5:
        shadow_hue = tf.random.uniform([], 0.4, 0.7); shadow_saturation = tf.random.uniform([], 0.0, 0.25)
        highlight_hue = tf.random.uniform([], 0.05, 0.15); highlight_saturation = tf.random.uniform([], 0.0, 0.25)
        params['shadow_hue'] = (shadow_hue - 0.4) / 0.3; params['shadow_sat'] = shadow_saturation / 0.25
        params['highlight_hue'] = (highlight_hue - 0.05) / 0.1; params['highlight_sat'] = highlight_saturation / 0.25
    else:
        params['shadow_hue'] = 0.5; params['shadow_sat'] = 0.0; params['highlight_hue'] = 0.5; params['highlight_sat'] = 0.0
    if tf.random.uniform([]) > 0.5:
        strength = tf.random.uniform([], 0.0, 0.05); params['grain_strength'] = strength / 0.05
        image = image + tf.random.normal(shape=tf.shape(image), stddev=strength)
    else: params['grain_strength'] = 0.0
    if tf.random.uniform([]) > 0.5:
        shape = tf.shape(image); y, x = tf.meshgrid(tf.linspace(-1.0, 1.0, shape[0]), tf.linspace(-1.0, 1.0, shape[1]))
        radius = tf.sqrt(x**2 + y**2)
        strength = tf.random.uniform([], 0.5, 1.5); params['vignette_strength'] = (strength - 0.5) / 1.0
        falloff = tf.random.uniform([], 0.5, 1.5); params['vignette_falloff'] = (falloff - 0.5) / 1.0
        image = image * tf.expand_dims(tf.clip_by_value(1.0 - strength * (radius ** falloff), 0.0, 1.0), axis=-1)
    else: params['vignette_strength'] = 0.0; params['vignette_falloff'] = 0.0
    image = tf.clip_by_value(image, 0.0, 1.0)
    param_vector = tf.stack([
        params['brightness'], params['saturation'], params['r_gain'], params['g_gain'], params['b_gain'],
        params['s_curve_contrast'], params['linear_contrast'], params['shadow_hue'], params['shadow_sat'],
        params['highlight_hue'], params['highlight_sat'], params['grain_strength'],
        params['vignette_strength'], params['vignette_falloff']
    ])
    return image, param_vector

def create_regression_training_pair_v2(image_file):
    neutral_image = load_image(image_file)
    filtered_image, param_vector = apply_and_return_params_v2(neutral_image)
    return neutral_image, filtered_image, param_vector

test_data = [create_regression_training_pair_v2(fp) for fp in test_filepaths]
neutral_images, filtered_images, true_params = zip(*test_data)

filtered_images_ds = tf.data.Dataset.from_tensor_slices(list(filtered_images)).batch(BATCH_SIZE)
true_params_np = np.array(true_params)

test_evaluation_ds = tf.data.Dataset.from_tensor_slices((list(filtered_images), true_params_np)).batch(BATCH_SIZE)

print(f"✅ Test set created with {len(test_filepaths)} samples.")

# TEST 1: QUANTITATIVE EVALUATION (OVERALL MSE)
print("\n--- Running Test 1: Quantitative Evaluation (Overall MSE) ---")
loss = model.evaluate(test_evaluation_ds, verbose=1)
print(f"\n✅ Final MSE Loss on Hold-Out Test Set: {loss:.6f}")

# TEST 2: PER-PARAMETER ACCURACY (MAE)
print("\n--- Running Test 2: Per-Parameter Accuracy (MAE) ---")
predicted_params_np = model.predict(filtered_images_ds, verbose=1)

# Calculate Mean Absolute Error for each parameter
per_parameter_mae = np.mean(np.abs(true_params_np - predicted_params_np), axis=0)

mae_df = pd.DataFrame({'Parameter': PARAM_ORDER, 'MAE': per_parameter_mae})
mae_df = mae_df.sort_values(by='MAE', ascending=False)

print("\nPer-Parameter Mean Absolute Error:")
print(mae_df)

# Generate and save the bar chart
plt.figure(figsize=(12, 8))
plt.bar(mae_df['Parameter'], mae_df['MAE'], color='skyblue')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Per-Parameter Prediction Accuracy on Test Set')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
bar_chart_path = os.path.join(RESULTS_DIR, 'per_parameter_mae_chart.png')
plt.savefig(bar_chart_path)
plt.close()
print(f"✅ Per-parameter MAE bar chart saved to: {bar_chart_path}")

# TEST 3 & 4: QUALITATIVE ANALYSIS (BEST/WORST/TYPICAL CASES)
print("\n--- Running Tests 3 & 4: Qualitative Analysis ---")

def apply_params_to_image(image, param_vector):
    """
    Takes a [0,1] image tensor of ANY size and applies the predicted filter.
    This is the "parameter decoder".
    """
    if hasattr(param_vector, 'numpy'):
        param_vector_np = param_vector.numpy()
    else:
        param_vector_np = param_vector

    params = dict(zip(PARAM_ORDER, param_vector_np))

    # De-normalize and apply each effect mathematically
    image = tf.image.adjust_brightness(image, params['brightness'] * 0.3 - 0.15)
    image = tf.image.adjust_saturation(image, params['saturation'] * 0.6 + 0.7)
    gains = tf.constant([params['r_gain'] * 0.2 + 0.9, params['g_gain'] * 0.2 + 0.9, params['b_gain'] * 0.2 + 0.9], dtype=tf.float32)
    image *= gains

    if params['s_curve_contrast'] > 0.01:
        factor = params['s_curve_contrast'] * 6.0 + 2.0
        image = tf.sigmoid(factor * (image - 0.5))
    elif params['linear_contrast'] > 0.51 or params['linear_contrast'] < 0.49:
        factor = params['linear_contrast'] * 0.4 + 0.8
        image = tf.image.adjust_contrast(image, factor)

    if params['grain_strength'] > 0.01:
        strength = params['grain_strength'] * 0.05
        image += tf.random.normal(shape=tf.shape(image), stddev=strength)

    if params['vignette_strength'] > 0.01:

        def apply_vignette_numpy(image_np, strength_param, falloff_param):
            height, width, _ = image_np.shape

            x_coords = np.linspace(-1.0, 1.0, width)
            y_coords = np.linspace(-1.0, 1.0, height)
            x, y = np.meshgrid(x_coords, y_coords)

            radius = np.sqrt(x**2 + y**2)

            strength = strength_param * 1.0 + 0.5
            falloff = falloff_param * 1.0 + 0.5

            vignette_mask = np.clip(1.0 - strength * (radius ** falloff), 0.0, 1.0)

            vignette_3_channel = np.stack([vignette_mask] * 3, axis=-1)

            return image_np * vignette_3_channel

        image = tf.py_function(
            func=apply_vignette_numpy,
            inp=[image, params['vignette_strength'], params['vignette_falloff']],
            Tout=tf.float32
        )

    return tf.clip_by_value(image, 0.0, 1.0)

sample_errors = np.mean(np.square(true_params_np - predicted_params_np), axis=1)
best_case_idx = np.argmin(sample_errors)
worst_case_idx = np.argmax(sample_errors)
typical_indices = np.random.choice(len(sample_errors), 3, replace=False)

analysis_cases = {
    'best_case': best_case_idx,
    'worst_case': worst_case_idx,
    'typical_case_1': typical_indices[0],
    'typical_case_2': typical_indices[1],
    'typical_case_3': typical_indices[2]
}

for name, idx in analysis_cases.items():
    # Get the data for this specific case
    neutral_img = neutral_images[idx]
    input_to_ai = filtered_images[idx]
    true_p = true_params[idx]
    pred_p = predicted_params_np[idx]

    # Generate the images for comparison
    ground_truth_filter = apply_params_to_image(neutral_img, true_p)
    ai_predicted_filter = apply_params_to_image(neutral_img, pred_p)

    # Create and save the 3-panel plot
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f'Qualitative Analysis: {name.replace("_", " ").title()}', fontsize=16)

    axs[0].imshow(ground_truth_filter); axs[0].set_title("Ground Truth Filter"); axs[0].axis('off')
    axs[1].imshow(ai_predicted_filter); axs[1].set_title("AI Predicted Filter"); axs[1].axis('off')
    axs[2].imshow(input_to_ai); axs[2].set_title("Input Image Analyzed by AI"); axs[2].axis('off')

    # Add parameter comparison text below the plot
    true_str = "True:      " + " ".join([f"{p:.2f}" for p in true_p])
    pred_str = "Predicted: " + " ".join([f"{p:.2f}" for p in pred_p])
    fig.text(0.5, 0.05, f"{true_str}\n{pred_str}", ha='center', fontsize=10, fontfamily='monospace')

    plt.tight_layout(rect=[0, 0.1, 1, 0.95])

    plot_path = os.path.join(RESULTS_DIR, f'qualitative_{name}.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"✅ Qualitative analysis plot saved for '{name}' to: {plot_path}")

print("\n--- All Report Generation Tests Complete! ---")
print(f"Check the '{RESULTS_DIR}' folder in your Google Drive for all generated figures.")

--- Starting Report Generation Script ---

Loading final trained model from: /content/drive/MyDrive/ImageFilterGenerator_Regression/final_regression_model.keras
✅ Model loaded successfully.
✅ Model recompiled for evaluation.

Creating a hold-out test set from COCO validation split...


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Sufficient images already downloaded


...

 100% |███████████████| 1000/1000 [5.4s elapsed, 0s remaining, 158.3 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [5.4s elapsed, 0s remaining, 158.3 samples/s]      


Dataset 'coco-2017-validation-1000' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-validation-1000' created


✅ Test set created with 1000 samples.

--- Running Test 1: Quantitative Evaluation (Overall MSE) ---
32/32 ━━━━━━━━━━━━━━━━━━━━ 16s 187ms/step - loss: 0.0524

✅ Final MSE Loss on Hold-Out Test Set: 0.053149

--- Running Test 2: Per-Parameter Accuracy (MAE) ---
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 281ms/step

Per-Parameter Mean Absolute Error:
            Parameter       MAE
10      highlight_sat  0.284425
8          shadow_sat  0.272931
4              b_gain  0.220532
1          saturation  0.212877
2              r_gain  0.212422
3              g_gain  0.192400
5    s_curve_contrast  0.189833
0          brightness  0.182566
6     linear_contrast  0.137992
7          shadow_hue  0.128167
9       highlight_hue  0.128045
11     grain_strength  0.087983
13   vignette_falloff  0.075541
12  vignette_strength  0.028558
✅ Per-parameter MAE bar chart saved to: /content/drive/MyDrive/ImageFilterGenerator_Regression/report_results/per_parameter_mae_chart.png

--- Running Tests 3 & 4: Qualitative Analy